In [3]:
import numpy as np
import math
from spatialmath import SE3
from spatialmath.base import e2h,h2e
from roboticstoolbox import *
from machinevisiontoolbox import CentralCamera #针孔相机模型
%matplotlib widget

In [4]:
cam = CentralCamera(f=[913.08825216e-6,917.24582554e-6],rho = 1e-6, imagesize=[640,480], pp=[327.50021778,228.7678789])
print(cam)

           Name: perspective [CentralCamera]
     pixel size: 1e-06 x 1e-06
     image size: 640 x 480
           pose: t = 0, 0, 0; rpy/yxz = 0°, 0°, 0°
   principal pt: [327.50021778 228.7678789 ]
   focal length: [0.00091309 0.00091725]



In [5]:
cam.K

array([[913.08825216,   0.        , 327.50021778],
       [  0.        , 917.24582554, 228.7678789 ],
       [  0.        ,   0.        ,   1.        ]])

In [6]:
P = np.array([0,-0.28,0])

In [7]:
#相机DH
DFbot1 = DHRobot(
    [
        RevoluteMDH(d=0.04145,qlim=np.array([-np.pi,np.pi])),
        RevoluteMDH(alpha=np.pi/2,qlim=np.array([-np.pi,np.pi])),
        RevoluteMDH(a=-0.08285,qlim=np.array([-np.pi,np.pi])),
        RevoluteMDH(a=-0.08285,qlim=np.array([-np.pi,np.pi])),
        RevoluteMDH(alpha=-np.pi/2,qlim=np.array([0,np.pi])),
        RevoluteMDH(a=0.05,d=0.06,qlim=np.array([-np.pi,np.pi]))
    ],
    name = "DFbot",
)

In [26]:
T1 = DFbot1.fkine([np.pi/2,-np.pi/6,0,np.pi,0,0])
SE3(T1)
#DFbot1.plot([np.pi/2,-np.pi/6,0,np.pi,0,0])

   0        -1         0         0         
  -0.866     0        -0.5      -0.2168    
   0.5       0        -0.866     0.09734   
   0         0         0         1         


In [9]:
#相机投影
p = cam.project_point(P,pose=T1)
print(p)
#cam.plot_point(P, pose=T1)

[[375.26034975]
 [228.7678789 ]]


In [10]:
from machinevisiontoolbox import mkcube #生成立方体
X,Y,Z = mkcube(s=0.03, centre = (0,-0.27,0.015),edge = True)
cam.plot_wireframe(X,Y,Z,pose=T1)

In [11]:
#立方体真实世界坐标
print("X",X)
print("Y",Y)
print("Z",Z)

X [[-0.015 -0.015  0.015  0.015 -0.015]
 [-0.015 -0.015  0.015  0.015 -0.015]]
Y [[-0.285 -0.255 -0.255 -0.285 -0.285]
 [-0.285 -0.255 -0.255 -0.285 -0.285]]
Z [[0.   0.   0.   0.   0.  ]
 [0.03 0.03 0.03 0.03 0.03]]


In [12]:
#使用空间点和平面像素点求解相机位姿

In [23]:
#整理点
object_points = []
image_points = []
for (x,y,z) in zip(X[0,0:4],Y[0,0:4],Z[0,0:4]):
    object_points.append((x,y,z))

print(object_points)

for p in object_points:
    P = cam.project_point(p,pose=T1)
    image_points.append((P[0][0],P[1][0]))

print(image_points)

object_points = np.array(object_points)
image_points = np.array(image_points)

[(-0.015, -0.28500000000000003, 0.0), (-0.015, -0.255, 0.0), (0.015, -0.255, 0.0), (0.015, -0.28500000000000003, 0.0)]
[(407.64626984628256, 344.9761957330334), (189.83925718412038, 361.83479817596447), (189.83925718412038, 95.70095962403549), (407.64626984628256, 112.55956206696656)]


In [28]:
import cv2
camera_matrix = cam.K
dist_coeffs = np.zeros((4,1)) #假设没有畸变

_,rvec,tvec = cv2.solvePnP(object_points,image_points,camera_matrix,dist_coeffs) #使用pnp求解外参

R,_ = cv2.Rodrigues(rvec) #将旋转向量转为旋转矩阵
extrinsic_matrix = np.hstack((R,tvec))
print("extrinsic matrix solved:\n",extrinsic_matrix)

extrinsic matrix solved:
 [[-1.83336335e-09 -8.66025445e-01  4.99999929e-01 -2.36425011e-01]
 [-1.00000000e+00 -3.17940585e-09 -9.17362114e-09 -8.70757381e-10]
 [ 9.53429211e-09 -4.99999929e-01 -8.66025445e-01 -2.41032258e-02]]


In [33]:
#外参矩阵应为T1的逆矩阵
print(np.linalg.inv(T1))

[[-8.36449319e-17 -8.66025404e-01  5.00000000e-01 -2.36425000e-01]
 [-1.00000000e+00  3.06161700e-17 -1.14261102e-16  4.73612267e-18]
 [ 8.36449319e-17 -5.00000000e-01 -8.66025404e-01 -2.41032470e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
